In [ ]:
import pandas as pd
import numpy as np
from PIL import Image

# データセットの読み込み

In [ ]:
df = pd.read_csv('kaonavi.csv', index_col='employee_id')

In [ ]:
df.head(5)

### memo
- input: image ( height: 120, width: 90)

# Generator

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D

def build_generator():
    input_shape = (100,)
    
    model = Sequential()

    model.add(Dense(20 * 15 * 128, activation="relu", input_shape=input_shape))
    model.add(Reshape((20, 15, 128)))
    model.add(BatchNormalization())
    model.add(UpSampling2D(size=3))

    model.add(Conv2D(32, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(UpSampling2D())
    
    model.add(Conv2D(1, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    
    return model
    
    # print(model.summary())
    # 
    # noise = Input(shape=input_shape)
    # img = model(noise)
    # return Model(noise, img)

In [ ]:
model = build_generator()
model.summary()

# Discriminator

In [ ]:
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout

def build_discriminator():
    input_shape = (120, 90, 1)
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, input_shape=input_shape, padding="same"))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
model = build_discriminator()
model.summary()

---

# モデル作り

In [ ]:
employee_ids = df[df.job_type=='エンジニア'].index
print('length of employee_ids: {}'.format(len(employee_ids)))

In [ ]:
BATCH_SIZE = 30
NUM_EPOCH = 200

In [ ]:
from keras.optimizers import Adam
from keras.layers import Input
from keras.models import Model

# モデルのコンパイル
optimizer = Adam(lr=0.0002, beta_1=0.5)

discriminator = build_discriminator()
d_opt = Adam(lr=1e-5, beta_1=0.1)
discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)

discriminator.trainable = False
generator = build_generator()
dcgan = Sequential([generator, discriminator])
g_opt = Adam(lr=2e-4, beta_1=0.5)
dcgan.compile(loss='binary_crossentropy', optimizer=g_opt)

# 画像の読み込み
images = [np.array(Image.open('faces_monochrome/{}.png'.format(i))).reshape(120, 90, -1) for i in employee_ids]

# 初期化とか
num_batches = int(len(images) / BATCH_SIZE)
test_noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
d_losses = []
g_losses = []

for epoch in range(NUM_EPOCH):
    for index in range(num_batches):
        noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
        image_batch = images[index * BATCH_SIZE:(index+1) * BATCH_SIZE]
        generated_images = generator.predict(noise)

        # update discriminator
        X = np.concatenate((np.array(image_batch), generated_images))
        y = [1]*BATCH_SIZE + [0]*BATCH_SIZE
        d_loss = discriminator.train_on_batch(X, y)
        d_losses.append(d_loss)

        # update generator
        noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
        g_loss = dcgan.train_on_batch(noise, np.ones((BATCH_SIZE, 1)))
        g_losses.append(g_loss)

    print("epoch: %d, g_loss: %f, d_loss: %f" % (epoch, g_loss, d_loss))
    res = Image.fromarray(np.array(generator.predict(test_noise)[0][:,:,0]*300, dtype='uint8'))
    res.save('generated_images/epoch_{}.png'.format(epoch))

# visualization
fig, (ax_d, ax_g) = plt.subplots(ncols=2, figsize=(10,4))
ax_d.plot(d_losses, linewidth=2)
ax_d.set_title('loss of discriminator')

ax_g.plot(g_losses, linewidth=2)
ax_g.set_title('loss of generator')
fig.show()